## Random functions

Random functions can be implemented in *Quibbler* using *random* function quibs. Random function quibs are function quibs that call random functions and are specified with (TODO: update is_random property). Defining a function quib as random forces the quib to cache its output, so that multiple request for its value give the same random realization (quenched randomness). Then, to refresh randomization, we invalidate the cached values of these random quibs, which can be done either centrally for all random quibs, or inidvidually for each given random quib. Such invalidation of random quibs will then invalidate any downstream calculations that depend on these random values, causing re-evaluated of the random function upon downstream output request.

### Import

In [14]:
import pyquibbler as qb
from pyquibbler import iquib, q, reset_impure_function_quibs
qb.override_all()
import numpy as np

### Quibs calling *NumPy* random functions are automatically defined as random quibs

By default, all standard *NumPy* functions that generate random output are automatically implemented as random function quibs. We can therefore define random quibs simply by calling *NumPy* random functions with quib arguments.

For example,

In [2]:
n = iquib(3)
rand_numbers = np.random.rand(n)
rand_numbers.get_value()

array([0.30530187, 0.27942297, 0.127104  ])

### Random quibs always cache their results

Random quibs always cache their results, so repeated calls for their value yields the same randomization:

In [21]:
rand_numbers.get_value()

array([0.74206872, 0.82927428, 0.39501332])

Because the randomization is fixed, mathematical trivialities hold true:

In [22]:
rand_numbers_plus_1 = rand_numbers + 1
should_equal_zero = np.sum(rand_numbers_plus_1 - rand_numbers) - n
should_equal_zero.get_value()

0.0

### Refreshing randomization

The cache of the random quibs can be invalidated either centrally for all random quibs, or individually for a given specific random quib. Upon invalidation, all downstream dependent quibs are also invalidated. Requesting the value of such downstream calculations will then lead to recalculation of the random function (re-randomization).

**Central re-randomization of all random quibs.** To simply refresh randomization of all the random quibs in an entire analysis pipeline, we use the `reset_impure_function_quibs` function (TODO: replace function name). All downstream results are also invalidated and upon request for their value, new randomization will be calculated:

In [23]:
rand_numbers_plus_1.get_value()

array([1.74206872, 1.82927428, 1.39501332])

In [24]:
reset_impure_function_quibs()
rand_numbers_plus_1.get_value()

array([1.35043564, 1.08631101, 1.80128479])

**Quib-specific re-randomization.** To specifically refresh the randomization of a given chosen random quib, we can invalidate its cache using the `reset_cache` method. Any function quibs downstream of this specific quib will thereby also invalidate. Request the value of such downstream results will lead to new randomization:

In [25]:
rand_numbers.reset_cache()
rand_numbers_plus_1.get_value()

array([1.71539653, 1.10610631, 1.27845747])

### Examples

For an example of an *Quibbler* app with random quibs, see:

* [[quibdemo_fft]] 
* [[quibdemo_random_quibs_dice]].